# Load packages and data

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir_path = '/content/drive/MyDrive/ML Project/data/image/'
train_tabular = pd.read_csv(data_dir_path + 'train_image_sample')
test_tabular = pd.read_csv( data_dir_path + 'test_image_sample')
print(train_tabular.shape)
print(test_tabular.shape)

(222532, 43)
(55635, 43)


In [ ]:
train_tabular_X = train_tabular.drop(['title','description', 'item_id', 'user_id', 'item_seq_number', 'activation_date', 'image',
        'image_top_1', 'deal_probability'], axis=1)
test_tabular_X = test_tabular.drop(['title','description', 'item_id', 'user_id', 'item_seq_number', 'activation_date', 'image',
        'image_top_1', 'deal_probability'], axis=1)
train_y = train_tabular['deal_probability']
test_y = test_tabular['deal_probability']

categorical_features = train_tabular_X.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_features:
    train_tabular_X[col] = train_tabular_X[col].astype('category')
    test_tabular_X[col] = test_tabular_X[col].astype('category')

print(train_tabular_X.shape)
print(test_tabular_X.shape)

(222532, 34)
(55635, 34)


In [ ]:
# keep only the svd embedding columns, and rows that are in the image subsample

# be very careful using merge: the one calling merge keeps the order!
train_text_X = pd.read_csv(data_dir_path + '../train_embeddings_fasttext_svd.csv')
train_text_X = train_tabular[["item_id", "user_id"]].merge(train_text_X, on=["item_id", "user_id"], how="inner")
train_text_X = train_text_X.drop(columns = train_tabular.columns.tolist(), errors='ignore')

test_text_X = pd.read_csv(data_dir_path + '../test_embeddings_fasttext_svd.csv')
test_text_X = test_tabular[["item_id", "user_id"]].merge(test_text_X, on=["item_id", "user_id"], how="inner")
test_text_X = test_text_X.drop(columns = test_tabular.columns.tolist(), errors='ignore')

print(train_text_X.shape)
print(test_text_X.shape)

(222532, 42)
(55635, 42)


# Helper functions

In [ ]:
# !pip install optuna

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from sklearn.decomposition import TruncatedSVD
import optuna

In [ ]:
def tune_params(train_X, train_y, categorical_features):
    train_data, val_data, train_label, val_label = train_test_split(
        train_X, train_y, test_size=0.2, random_state=42)

    def objective(trial):
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            'num_leaves': trial.suggest_int('num_leaves', 32, 512),
            'max_depth': trial.suggest_int('max_depth', 3, 25),
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10.0),
            'verbose': -1,
            'bagging_seed': 42,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'feature_fraction': 0.8,
        }

        dtrain = lgb.Dataset(train_data, label=train_label, categorical_feature=categorical_features)
        dval = lgb.Dataset(val_data, label=val_label, reference=dtrain, categorical_feature=categorical_features)

        callbacks = [
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
        model = lgb.train(
            params, dtrain, valid_sets=[dtrain, dval], callbacks=callbacks,
            num_boost_round=1000
        )
        preds = model.predict(val_data)
        return root_mean_squared_error(val_label, preds)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50, timeout=5400)

    print("Best parameters:", study.best_params)

In [ ]:
def fit_lgb(
        train_X, train_y, test_X, test_y, categorical_features,
        params,
        num_boost_round=3000,
        early_stopping_rounds=50,
        eval_period=40,
    ):

    # validation set for early stopping
    train_X_split, val_X_split, train_y_split, val_y_split = train_test_split(
        train_X, train_y, test_size=0.2, random_state=42, shuffle=True
    )
    lgb_train = lgb.Dataset(
        train_X_split, label=train_y_split, categorical_feature=categorical_features
    )
    lgb_val = lgb.Dataset(
        val_X_split, label=val_y_split, categorical_feature=categorical_features,
        reference=lgb_train
    )

    callbacks = [
        lgb.early_stopping(stopping_rounds=early_stopping_rounds),
        lgb.log_evaluation(period=eval_period)
    ]

    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=num_boost_round,
        valid_sets=[lgb_train, lgb_val],
        callbacks=callbacks
    )

    preds = model.predict(test_X, num_iteration=model.best_iteration)
    rmse = root_mean_squared_error(test_y, preds)
    print(f"RMSE: {rmse}")

    return model

In [ ]:
def get_feature_importance(model):
    feature_importances = pd.DataFrame({
        'feature': model.feature_name(),
        'importance': model.feature_importance()
    }).sort_values(by='importance', ascending=False)

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(feature_importances)
    return feature_importances

# Tabular tree

In [ ]:
# This param is tuned in another notebook for subset tabular tree
tabular_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.015,
    'num_leaves': 512,
    'max_depth': 19,
    'bagging_seed': 42,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'feature_fraction': 0.8,
    'lambda_l1': 0.004,
    'lambda_l2': 1.41,
    'verbose': -1
}

In [ ]:
model_tabular = fit_lgb(
    train_tabular_X, train_y,
    test_tabular_X, test_y,
    categorical_features=categorical_features,
    params=tabular_params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.234383	valid_1's rmse: 0.241233
[80]	training's rmse: 0.221675	valid_1's rmse: 0.234866
[120]	training's rmse: 0.214027	valid_1's rmse: 0.232811
[160]	training's rmse: 0.208875	valid_1's rmse: 0.232367
[200]	training's rmse: 0.205061	valid_1's rmse: 0.232388
Early stopping, best iteration is:
[170]	training's rmse: 0.207704	valid_1's rmse: 0.232313
RMSE: 0.23201362973381603


# Embedding Tree

In [ ]:
tune_params(train_text_X, train_y, categorical_features=[])

[I 2024-12-03 04:16:40,866] A new study created in memory with name: no-name-bb83d0f3-b9db-48d5-9527-653f662e7715


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.240926	valid_1's rmse: 0.244877
[200]	training's rmse: 0.234531	valid_1's rmse: 0.242074
[300]	training's rmse: 0.230211	valid_1's rmse: 0.241069
[400]	training's rmse: 0.226788	valid_1's rmse: 0.240616
[500]	training's rmse: 0.223784	valid_1's rmse: 0.240428
[600]	training's rmse: 0.22109	valid_1's rmse: 0.240287
[700]	training's rmse: 0.218498	valid_1's rmse: 0.240175
[800]	training's rmse: 0.216038	valid_1's rmse: 0.24007
[900]	training's rmse: 0.213775	valid_1's rmse: 0.239996
[1000]	training's rmse: 0.211495	valid_1's rmse: 0.239891
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.211495	valid_1's rmse: 0.239891


[I 2024-12-03 04:16:56,832] Trial 0 finished with value: 0.2398908645834304 and parameters: {'learning_rate': 0.017440585143575008, 'num_leaves': 100, 'max_depth': 21, 'lambda_l1': 0.007710398526450154, 'lambda_l2': 1.8217059836820577}. Best is trial 0 with value: 0.2398908645834304.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.243774	valid_1's rmse: 0.245542
[200]	training's rmse: 0.240695	valid_1's rmse: 0.243691
[300]	training's rmse: 0.23883	valid_1's rmse: 0.242847
[400]	training's rmse: 0.237479	valid_1's rmse: 0.242362
[500]	training's rmse: 0.236264	valid_1's rmse: 0.242061
[600]	training's rmse: 0.23515	valid_1's rmse: 0.241803
[700]	training's rmse: 0.234042	valid_1's rmse: 0.241535
[800]	training's rmse: 0.233059	valid_1's rmse: 0.241334
[900]	training's rmse: 0.232147	valid_1's rmse: 0.24119
[1000]	training's rmse: 0.231154	valid_1's rmse: 0.241025
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.231154	valid_1's rmse: 0.241025


[I 2024-12-03 04:17:07,045] Trial 1 finished with value: 0.24102523941516207 and parameters: {'learning_rate': 0.027730747136183554, 'num_leaves': 100, 'max_depth': 5, 'lambda_l1': 0.003513079130919596, 'lambda_l2': 0.24932944307509602}. Best is trial 0 with value: 0.2398908645834304.


...

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.23694	valid_1's rmse: 0.244626
[200]	training's rmse: 0.227198	valid_1's rmse: 0.241323
[300]	training's rmse: 0.220286	valid_1's rmse: 0.240149
[400]	training's rmse: 0.214385	valid_1's rmse: 0.239546
[500]	training's rmse: 0.209062	valid_1's rmse: 0.239187
[600]	training's rmse: 0.204088	valid_1's rmse: 0.238929
[700]	training's rmse: 0.199425	valid_1's rmse: 0.238772
[800]	training's rmse: 0.19506	valid_1's rmse: 0.238656
[900]	training's rmse: 0.19091	valid_1's rmse: 0.238601
[1000]	training's rmse: 0.186956	valid_1's rmse: 0.238561
Did not meet early stopping. Best iteration is:
[996]	training's rmse: 0.187109	valid_1's rmse: 0.238552


[I 2024-12-03 04:38:15,617] Trial 48 finished with value: 0.23855187781410242 and parameters: {'learning_rate': 0.014850285341428824, 'num_leaves': 396, 'max_depth': 23, 'lambda_l1': 5.549345130605934, 'lambda_l2': 0.021055411264221675}. Best is trial 48 with value: 0.23855187781410242.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.234162	valid_1's rmse: 0.243324
[200]	training's rmse: 0.223626	valid_1's rmse: 0.240543
[300]	training's rmse: 0.215895	valid_1's rmse: 0.239633
[400]	training's rmse: 0.20917	valid_1's rmse: 0.23922
[500]	training's rmse: 0.20303	valid_1's rmse: 0.238955
[600]	training's rmse: 0.19736	valid_1's rmse: 0.238764
[700]	training's rmse: 0.192081	valid_1's rmse: 0.238705
[800]	training's rmse: 0.187097	valid_1's rmse: 0.238665
[900]	training's rmse: 0.182415	valid_1's rmse: 0.238633
[1000]	training's rmse: 0.178026	valid_1's rmse: 0.238622
Did not meet early stopping. Best iteration is:
[959]	training's rmse: 0.179783	valid_1's rmse: 0.238588


[I 2024-12-03 04:39:01,847] Trial 49 finished with value: 0.23858802672695492 and parameters: {'learning_rate': 0.018318519704118606, 'num_leaves': 402, 'max_depth': 23, 'lambda_l1': 5.548775942371967, 'lambda_l2': 0.018356135723679712}. Best is trial 48 with value: 0.23855187781410242.


Best parameters: {'learning_rate': 0.014850285341428824, 'num_leaves': 396, 'max_depth': 23, 'lambda_l1': 5.549345130605934, 'lambda_l2': 0.021055411264221675}


In [ ]:
text_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.015,
    'num_leaves': 396,
    'max_depth': 23,
    'lambda_l1': 5.549345130605934,
    'lambda_l2': 0.021055411264221675,
    'verbose': -1,
    'bagging_seed': 42,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'feature_fraction': 0.8,
}

In [ ]:
model_text = fit_lgb(
    train_text_X, train_y,
    test_text_X, test_y,
    categorical_features=[],
    params=text_params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.247238	valid_1's rmse: 0.250612
[80]	training's rmse: 0.239611	valid_1's rmse: 0.245883
[120]	training's rmse: 0.23443	valid_1's rmse: 0.243499
[160]	training's rmse: 0.230409	valid_1's rmse: 0.242128
[200]	training's rmse: 0.227032	valid_1's rmse: 0.241274
[240]	training's rmse: 0.224059	valid_1's rmse: 0.240686
[280]	training's rmse: 0.221365	valid_1's rmse: 0.240284
[320]	training's rmse: 0.218833	valid_1's rmse: 0.239979
[360]	training's rmse: 0.216443	valid_1's rmse: 0.239741
[400]	training's rmse: 0.214164	valid_1's rmse: 0.239556
[440]	training's rmse: 0.211958	valid_1's rmse: 0.239398
[480]	training's rmse: 0.209837	valid_1's rmse: 0.23924
[520]	training's rmse: 0.207776	valid_1's rmse: 0.239121
[560]	training's rmse: 0.20576	valid_1's rmse: 0.23899
[600]	training's rmse: 0.203823	valid_1's rmse: 0.238921
[640]	training's rmse: 0.20191	valid_1's rmse: 0.238894
[680]	training's rmse: 0.200028	va

# Tabular + text

In [ ]:
train_merge_X = pd.concat([train_tabular_X, train_text_X], axis=1)
test_merge_X = pd.concat([test_tabular_X, test_text_X], axis=1)

In [ ]:
tune_params(train_merge_X, train_y, categorical_features=categorical_features)

[I 2024-12-03 05:57:39,096] A new study created in memory with name: no-name-ff271f54-0f61-4e77-8c00-cdf1e4c8cee9


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.202132	valid_1's rmse: 0.23193
Early stopping, best iteration is:
[66]	training's rmse: 0.207337	valid_1's rmse: 0.231688


[I 2024-12-03 05:57:46,880] Trial 0 finished with value: 0.2316882733387526 and parameters: {'learning_rate': 0.05310275110180667, 'num_leaves': 448, 'max_depth': 18, 'lambda_l1': 5.51583733181019, 'lambda_l2': 0.00932741871370131}. Best is trial 0 with value: 0.2316882733387526.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.232839	valid_1's rmse: 0.236998
[200]	training's rmse: 0.225528	valid_1's rmse: 0.233224
[300]	training's rmse: 0.222554	valid_1's rmse: 0.232626
[400]	training's rmse: 0.220648	valid_1's rmse: 0.232447
[500]	training's rmse: 0.218956	valid_1's rmse: 0.232321
Early stopping, best iteration is:
[505]	training's rmse: 0.218844	valid_1's rmse: 0.232297


[I 2024-12-03 05:57:58,982] Trial 1 finished with value: 0.23229652978358856 and parameters: {'learning_rate': 0.011013300206330389, 'num_leaves': 99, 'max_depth': 7, 'lambda_l1': 0.0032528774230034985, 'lambda_l2': 0.002820449861445446}. Best is trial 0 with value: 0.2316882733387526.


...

Training until validation scores don't improve for 50 rounds


[I 2024-12-03 06:14:23,520] Trial 48 finished with value: 0.23157512817474588 and parameters: {'learning_rate': 0.06562507321008297, 'num_leaves': 358, 'max_depth': 23, 'lambda_l1': 1.1800629258096362, 'lambda_l2': 0.003378454941973053}. Best is trial 39 with value: 0.2306044397095248.


Early stopping, best iteration is:
[46]	training's rmse: 0.202708	valid_1's rmse: 0.231575
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.21626	valid_1's rmse: 0.233898
[200]	training's rmse: 0.198553	valid_1's rmse: 0.231081
[300]	training's rmse: 0.187762	valid_1's rmse: 0.230922
Early stopping, best iteration is:
[317]	training's rmse: 0.186169	valid_1's rmse: 0.230898


[I 2024-12-03 06:14:55,040] Trial 49 finished with value: 0.23089783733658353 and parameters: {'learning_rate': 0.012592957264390092, 'num_leaves': 454, 'max_depth': 24, 'lambda_l1': 0.006188355905427857, 'lambda_l2': 0.0026706845968832303}. Best is trial 39 with value: 0.2306044397095248.


Best parameters: {'learning_rate': 0.011523347585691509, 'num_leaves': 481, 'max_depth': 25, 'lambda_l1': 0.002767282719496572, 'lambda_l2': 0.0049387238144752085}


In [ ]:
merged_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.012,
    'num_leaves': 481,
    'max_depth': 25,
    'lambda_l1': 0.002767282719496572,
    'lambda_l2': 0.0049387238144752085,
    'verbose': -1,
    'bagging_seed': 42,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'feature_fraction': 0.8,
}

In [ ]:
model_merged = fit_lgb(
    train_merge_X, train_y,
    test_merge_X, test_y,
    categorical_features=categorical_features,
    params=merged_params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.236242	valid_1's rmse: 0.243496
[80]	training's rmse: 0.221867	valid_1's rmse: 0.236109
[120]	training's rmse: 0.212155	valid_1's rmse: 0.232961
[160]	training's rmse: 0.204772	valid_1's rmse: 0.231628
[200]	training's rmse: 0.198526	valid_1's rmse: 0.231105
[240]	training's rmse: 0.193466	valid_1's rmse: 0.230943
[280]	training's rmse: 0.189173	valid_1's rmse: 0.230812
[320]	training's rmse: 0.185323	valid_1's rmse: 0.230816
Early stopping, best iteration is:
[286]	training's rmse: 0.188563	valid_1's rmse: 0.2308
RMSE: 0.2308630907772972
